In [2]:
import csv
import base64
import time
import fnmatch
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

In [3]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [4]:
load_dotenv()
g = Github(getenv("GITHUB"))

In [5]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Appropedia - Shares knowledge to build rich, sustainable lives. [https://www.appropedia.org/Welcome_to_Appropedia] - Awesome Clean Tech - Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities. [https://github.com/nglgzz/awesome-clean-tech] - Awesome Open Climate Science - Awesome Open Atmospheric, Ocean, and Climate Science. [https://github.com/pangeo-data/awesome-open-climate-science] - Awesome Earth - "What can I do about the climate crisis?" Here are 326 things you can do. [https://github.com/philsturgeon/awesome-earth] - Awesome Open Geoscience - Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome. [https://github.com/softwareunderground/awesome-open-geoscience] - Datasets APIs and open source projects related to Climate Change - A curated list of 

In [ ]:
csv_github_projects = open("./csv/github_projects.csv", "w", newline="")
csv_github_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "topics",
    "rubric",
    "last_update",
    "stargazers_count",
    "project_active_checkbox",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "created",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "open_pullrequests",
    "last_released_date",
    "last_release_tag_name",
    "community_development_distribution_score",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "organization_avatar"
]
writer_github_projects = csv.DictWriter(
    csv_github_projects, fieldnames=csv_github_fieldnames
)
writer_github_projects.writeheader()

csv_github_organizations = open("./csv/github_organizations.csv", "w", newline="")
csv_github_organizations_fieldnames = [
    "organization_name",
    "organization_user_name",
    "organization_url",
    "organization_location",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
]

                            
writer_github_organizations = csv.DictWriter(
    csv_github_organizations, fieldnames=csv_github_organizations_fieldnames
)
writer_github_organizations.writeheader()
organization_list = []

csv_gitlab_projects = open("./csv/gitlab_projects.csv", "w", newline="")
csv_gitlab_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_gitlab_projects = csv.DictWriter(
    csv_gitlab_projects, fieldnames=csv_gitlab_fieldnames
)
writer_gitlab_projects.writeheader()

csv_custom_platform_projects = open(
    "./csv/custom_platform_projects.csv", "w", newline=""
)
csv_custom_platform_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_custom_platform_projects = csv.DictWriter(
    csv_custom_platform_projects, fieldnames=csv_custom_platform_fieldnames
)
writer_custom_platform_projects.writeheader()


for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            try:

                remaining, limit = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 200
                if remaining < min_requests:
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp()
                    countdown((int(resettime) - int(current_time) + 2))

                print(
                    "GitHub Requests | Limit: {}, Remaining: {}".format(
                        limit, remaining
                    )
                )
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                user, project_name = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                
                # Check if the project is still active
                closed_issues = repo.get_issues(state="closed")
                open_issues = repo.get_issues(state="open")

                closed_pullrequests = repo.get_pulls(state="closed")
                open_pullrequests = repo.get_pulls(state="open")

                ## preprocessing
                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed

                # Do not process inactive projects
                if (
                    last_issue_closed_timedelta > timedelta(days=365)
                    and total_commits_last_year == 0
                    or repo.archived
                ):
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else:
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"

                try:
                    license = repo.get_license()
                    if license.license.spdx_id == "NOASSERTION":
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"

                labels = ",".join([entry.name for entry in repo.get_labels()])
                topics = ",".join(repo.get_topics())

                languages_states = repo.get_languages()
                programming_languages = ",".join(languages_states.keys())
                
                try:
                    dominating_language = list(languages_states.keys())[0]
                except:
                    dominating_language = ""

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:
                        if file.path == ".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print("No funding information found")
                    donation_platforms = None
                    accepts_donations = "false"

                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                except Exception as e:
                    print(e)

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(
                    sorted(contributor_activity.items(), key=lambda item: item[1])
                )
                weighted_contribution = {
                    k: v / commits_total for k, v in sorted_contributor.items()
                }

                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(
                    weighted_contribution.values()
                )

                try:
                    last_released_date = releases[0].published_at.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    )
                    last_release_tag_name = releases[0].tag_name

                except Exception as e:
                    print("No Release found")
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)

                total_number_of_commits = commits.totalCount

                # Gathering organization data
                if repo.organization is None:
                    organization_user_name = None

                elif repo.organization.login not in organization_list:
                    organization_user_name = repo.organization.login
                    organization_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    organization_public_repos = repo.organization.public_repos
                    organization_location = repo.organization.location
                    organization_url = repo.organization.html_url
                    organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                    organization_repos = g.search_repositories(query='user:'+organization_user_name, sort='updated')     
                    organization_last_update = organization_repos[0].updated_at.strftime("%Y/%m/%d, %H:%M:%S")
                    
                    for project in organization_repos:
                        print(project)
                        
                    print("Last Update")
                    print(organization_repos[0])
                        
                    organization_data = {
                        "organization_name": organization_name,
                        "organization_user_name":organization_user_name,
                        "organization_url":organization_url,
                        "organization_avatar": organization_avatar,
                        "organization_location": organization_location,
                        "organization_public_repos": organization_public_repos,
                        "organization_created": organization_created,
                        "organization_last_update": organization_last_update
                    }

                    organization_list.append(organization_user_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)

                else:
                    organization_name = repo.organization.login
                    organization_user_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    

                project_data = {
                    "project_name": entry.name,
                    "git_namespace": user,
                    "git_url": repo.clone_url,
                    "rubric": r.key,
                    "oneliner": entry.text[2:],
                    "topics": topics,
                    "organization": organization_name,
                    "organization_user_name": organization_user_name,
                    "created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "last_update": repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "total_commits_last_year": total_commits_last_year,
                    "project_active_checkbox": project_active_checkbox,
                    "last_issue_closed": last_issue_closed.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    ),
                    "last_released_date": last_released_date,
                    "last_release_tag_name": last_release_tag_name,
                    "total_number_of_commits": total_number_of_commits,
                    "community_development_distribution_score": community_development_distribution_score,
                    "stargazers_count": repo.stargazers_count,
                    "dominating_language": dominating_language,
                    "languages": programming_languages,
                    "homepage": repo.homepage,
                    "closed_issues": closed_issues.totalCount,
                    "open_issues": open_issues.totalCount,
                    "closed_pullrequests": closed_pullrequests.totalCount,
                    "open_pullrequests": open_pullrequests.totalCount,
                    "good_first_issue": repo.get_issues(
                        state="open", labels=["good first issue"]
                    ).totalCount,
                    "license": license_name,
                    "contributors": repo.get_contributors().totalCount,
                    "accepts_donations": accepts_donations,
                    "donation_platforms": donation_platforms,
                    "code_of_conduct": code_of_conduct,
                    "contribution_guide": contribution_guide,
                    "organization_avatar":organization_avatar
                }

                print("Project Data:")
                print(project_data)
                writer_github_projects.writerow(project_data)

            except Exception as e:
                print(colored("Failed to gather project information:"))
                print(colored(e, "red"))
                print(traceback.format_exc())

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
            }
            print("Project Data:")
            print(project_data)
            writer_gitlab_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
            }

            print("Project Data:")
            print(project_data)
            writer_custom_platform_projects.writerow(project_data)

csv_github_projects.close()
csv_github_organizations.close()
csv_gitlab_projects.close()
csv_custom_platform_projects.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 2302
pvlib-python is an active project
Code of conduct found
Contribution guide found
Repository(full_name="pvlib/pvlib-python")
Repository(full_name="pvlib/pvanalytics")
Repository(full_name="pvlib/pvmodules")
Repository(full_name="pvlib/pvsc2015")
Last Update
Repository(full_name="pvlib/pvlib-python")
Organization Data:
{'organization_name': 'pvlib', 'organization_user_name': 'pvlib', 'organization_url': 'https://github.com/pvlib', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_location': None, 'organization_public_repos': 4, 'organization_created': '2015/02/17, 00:04:09', 'organization_last_update': '2021/11/18, 06:27:37'}
Project Data:
{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics

------------------------
Processing rdtools
URL: https://github.com/NREL/rdtools
rdtools is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 2140
rdtools is an active project
Code of conduct found
Contribution guide found
Repository(full_name="NREL/EnergyPlusBuildResults")
Repository(full_name="NREL/SAM")
Repository(full_name="NREL/ssc")
Repository(full_name="NREL/EnergyPlus")
Repository(full_name="NREL/nfp")
Repository(full_name="NREL/REopt_Lite_API")
Repository(full_name="NREL/OPFLearn.jl")
Repository(full_name="NREL/BAR_Designs")
Repository(full_name="NREL/resstock")
Repository(full_name="NREL/eere_app_bootstrap")
Repository(full_name="NREL/EnergyIntensityIndicators")
Repository(full_name="NREL/thesource_bootstrap")
Repository(full_name="NREL/EnergyPlusRegressionTool")
Repository(full_name="NREL/PyDSS")
Repository(full_name="NREL/PowerGridworld")
Repository(full_name="NREL/OpenStudio-HPXML")
Repository(full_name="NREL/farms")
Repository(full_name="NREL/developer.nrel.gov")

Repository(full_name="NREL/lcoc-ldevs")
Repository(full_name="NREL/dencity-web")
Repository(full_name="NREL/PSSE_Network_Reduction")
Repository(full_name="NREL/SLiDEData")
Repository(full_name="NREL/MPRL")
Repository(full_name="NREL/reV_Dashboard")
Repository(full_name="NREL/ml-combustion-pdf-models")
Repository(full_name="NREL/GEAR-SCouP")
Repository(full_name="NREL/EVOLVE")
Repository(full_name="NREL/turbine-models")
Repository(full_name="NREL/bifacialvf")
Repository(full_name="NREL/usbuildingtypology")
Repository(full_name="NREL/SOWFA")
Repository(full_name="NREL/OpenStudio-analysis-gem")
Repository(full_name="NREL/Caltalyst-Layer-Crack-Detector-CLCD")
Repository(full_name="NREL/GHGRP-for-Energy")
Repository(full_name="NREL/sensor-impact-evaluation-and-verification")
Repository(full_name="NREL/EULP.github.io")
Repository(full_name="NREL/DER-Dispatch")
Repository(full_name="NREL/Solar-Forecasting")
Repository(full_name="NREL/Grid-Forecasting")
Repository(full_name="NREL/openstudio-me

Repository(full_name="NREL/eere-sep-projects-map")
Repository(full_name="NREL/waterfootprint")
Repository(full_name="NREL/ie-data-collection-animation")
Repository(full_name="NREL/omnibus-api-umbrella")
Repository(full_name="NREL/api-umbrella-web")
Repository(full_name="NREL/api-umbrella-gatekeeper")
Repository(full_name="NREL/api-umbrella-config")
Repository(full_name="NREL/api-umbrella-router")
Repository(full_name="NREL/BACnet")
Repository(full_name="NREL/DAKOTA_plugin")
Repository(full_name="NREL/BACnet-state")
Repository(full_name="NREL/HystepAnalysis")
Repository(full_name="NREL/eere-govdelivery-subscription-api")
Repository(full_name="NREL/dencity-scripts")
Repository(full_name="NREL/incubatenergy.org")
Repository(full_name="NREL/vagrant-boxes")
Repository(full_name="NREL/hpxml2reso")
Repository(full_name="NREL/solar-decathlon")
Repository(full_name="NREL/streamm-BuildingBlocks")
Repository(full_name="NREL/CampusEnergyModeling")
Repository(full_name="NREL/streamm-opv-project")
R

{'project_name': 'feedinlib', 'git_namespace': 'oemof', 'git_url': 'https://github.com/oemof/feedinlib.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Contains implementations of photovoltaic models to calculate electricity generation from a PV installation based on given solar radiation. Furthermore it contains all necessary pre-calculations.', 'topics': '', 'organization': 'oemof community', 'organization_user_name': 'oemof', 'created': '2014/07/24, 09:03:29', 'last_update': '2021/11/10, 15:29:20', 'total_commits_last_year': 52, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/08, 13:45:48', 'last_released_date': '2019/01/31, 14:19:59', 'last_release_tag_name': 'v0.0.12', 'total_number_of_commits': 502, 'community_development_distribution_score': 0.6940639269406392, 'stargazers_count': 64, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': '', 'closed_issues': 51, 'open_issues': 19, 'closed_pullrequests': 18, 'open_pullrequests': 4, 'go

...

{'project_name': 'Pysolar', 'git_namespace': 'pingswept', 'git_url': 'https://github.com/pingswept/pysolar.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A collection of Python libraries for simulating the irradiation of any point on earth by the sun. It includes code for extremely precise ephemeris calculations.', 'topics': '', 'organization': 'Breaking Bytes', 'organization_user_name': None, 'created': '2008/03/01, 23:35:48', 'last_update': '2021/11/08, 11:46:42', 'total_commits_last_year': 14, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/28, 19:24:20', 'last_released_date': '2021/09/20, 21:33:11', 'last_release_tag_name': '0.10', 'total_number_of_commits': 343, 'community_development_distribution_score': 0.4196891191709845, 'stargazers_count': 275, 'dominating_language': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Python', 'homepage': 'http://pysolar.org', 'closed_issues': 129, 'open_issues': 16, 'closed_pullrequests': 69, 'open_pullrequest

{'project_name': 'pv-system-profiler', 'git_namespace': 'slacgismo', 'git_url': 'https://github.com/slacgismo/pv-system-profiler.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Estimating PV array location and orientation from real-world power datasets.', 'topics': '', 'organization': 'slacgismo', 'organization_user_name': 'SLAC GISMo', 'created': '2019/04/23, 18:33:36', 'last_update': '2021/10/15, 10:45:29', 'total_commits_last_year': 425, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/13, 19:42:00', 'last_released_date': '2021/10/13, 19:42:35', 'last_release_tag_name': 'v0.1.4', 'total_number_of_commits': 778, 'community_development_distribution_score': 0.19581749049429653, 'stargazers_count': 6, 'dominating_language': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Python,Shell', 'homepage': None, 'closed_issues': 29, 'open_issues': 4, 'closed_pullrequests': 28, 'open_pullrequests': 1, 'good_first_issue': 0, 'license': 'BSD-2-Clause', 'contributor

Repository(full_name="google/it-cert-automation-practice")
Repository(full_name="google/boringssl")
Repository(full_name="google/ground-platform")
Repository(full_name="google/auto")
Repository(full_name="google/distbench")
Repository(full_name="google/starthinker")
Repository(full_name="google/capirca")
Repository(full_name="google/gson")
Repository(full_name="google/XNNPACK")
Repository(full_name="google/ChromeBrowserEnterprise")
Repository(full_name="google/flatbuffers")
Repository(full_name="google/triage-party")
Repository(full_name="google/rust-scudo")
Repository(full_name="google/go-tpm-tools")
Repository(full_name="google/j2objc")
Repository(full_name="google/neural-tangents")
Repository(full_name="google/benchmark")
Repository(full_name="google/trillian-examples")
Repository(full_name="google/seqio")
Repository(full_name="google/closure-templates")
Repository(full_name="google/uncertainty-baselines")
Repository(full_name="google/agi")
Repository(full_name="google/tsunami-secur

Repository(full_name="google/microscopeimagequality")
Repository(full_name="google/ml-compiler-opt")
Repository(full_name="google/tacotron")
Repository(full_name="google/oboe")
Repository(full_name="google/hctr2")
Repository(full_name="google/CommonLoopUtils")
Repository(full_name="google/UIforETW")
Repository(full_name="google/certificate-transparency-community-site")
Repository(full_name="google/mdl-ref-apps")
Repository(full_name="google/gousb")
Repository(full_name="google/pkcs11test")
Repository(full_name="google/cloud-forensics-utils")
Repository(full_name="google/pprof-nodejs")
Repository(full_name="google/gazoo-device")
Repository(full_name="google/GoogleDataTransport")
Repository(full_name="google/cel-go")
Repository(full_name="google/incremental-dom")
Repository(full_name="google/serde_json5")
Repository(full_name="google/picatrix")
Repository(full_name="google/android-auto-companion-android")
Repository(full_name="google/android-auto-companion-ios")
Repository(full_name="goo

{'project_name': 'nalu-wind', 'git_namespace': 'Exawind', 'git_url': 'https://github.com/Exawind/nalu-wind.git', 'rubric': 'Wind Turbines', 'oneliner': 'Solver for wind farm simulations targeting exascale computational platforms.', 'topics': 'low-mach,wind-energy,les,cfd,ecp,exascale-computing,exawind', 'organization': 'exawind', 'organization_user_name': 'Exawind', 'created': '2018/05/03, 15:39:32', 'last_update': '2021/11/18, 23:51:04', 'total_commits_last_year': 123, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/18, 22:25:00', 'last_released_date': '2021/03/21, 18:48:03', 'last_release_tag_name': 'v1.3.0', 'total_number_of_commits': 2132, 'community_development_distribution_score': 0.7732558139534884, 'stargazers_count': 66, 'dominating_language': 'C', 'languages': 'C,C++,Fortran,CMake,Shell,Python,Dockerfile', 'homepage': 'https://exawind.org', 'closed_issues': 883, 'open_issues': 31, 'closed_pullrequests': 684, 'open_pullrequests': 6, 'good_first_issue': 0, 'lic